# Original Code

In [ ]:
"""
Product Description Generator - STARTER CODE (Needs Refactoring)
This code works but has many issues that need to be fixed.
"""

import json
from openai import OpenAI
from pydantic import BaseModel, Field, validator
from typing import List, Optional

class Product(BaseModel):
    id: str
    name: str
    category: str
    price: float
    features: List[str] = []
    
    @validator('price')
    def price_must_be_positive(cls, v):
        if v <= 0:
            raise ValueError('Price must be positive')
        return v

def generate_product_descriptions(json_file):
    # Load JSON file
    with open(json_file, 'r') as f:
        data = json.load(f)
    
    # Validate products
    products = []
    for item in data.get('products', []):
        try:
            product = Product(**item)
            products.append(product)
        except:
            pass  # Silent failure!
    
    # Generate descriptions
    client = OpenAI(api_key="your-api-key-here")
    results = []
    
    for product in products:
        # Create prompt
        prompt = f"""Create a product description for:
Name: {product.name}
Category: {product.category}
Price: ${product.price}
Features: {', '.join(product.features)}

Generate a compelling product description."""
        
        # Call API
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}]
        )
        
        # Process response
        description = response.choices[0].message.content
        results.append({
            "product_id": product.id,
            "name": product.name,
            "description": description
        })
    
    # Save results
    with open('results.json', 'w') as f:
        json.dump(results, f, indent=2)
    
    return results

# Usage
if __name__ == "__main__":
    generate_product_descriptions("products.json")


# Create Json sample file

In [4]:
import json

products_data = {
    "products": [
        {
            "id": "P001",
            "name": "Wireless Bluetooth Headphones",
            "category": "Electronics",
            "price": 99.99,
            "features": ["Bluetooth 5.0", "Noise Cancelling", "30hr Battery", "Comfortable Fit"]
        },
        {
            "id": "P002",
            "name": "Smart Watch",
            "category": "Wearables",
            "price": 249.99,
            "features": ["Heart Rate Monitor", "GPS", "Water Resistant", "Sleep Tracking"]
        },
        {
            "id": "P003",
            "name": "Laptop Stand",
            "category": "Accessories",
            "price": 49.99,
            "features": ["Adjustable Height", "Aluminum Construction", "Cable Management"]
        }
    ]
}

with open("products.json", "w", encoding="utf-8") as f:
    json.dump(products_data, f, indent=2)

print("products.json file created successfully.")


products.json file created successfully.


In [5]:
import json

invalid_products_data = {
    "products": [
        {
            "id": "P100",
            "name": "Invalid Negative Price",
            "category": "Electronics",
            "price": -10.99,  # ❌ Negative price
            "features": ["Feature A"]
        },
        {
            "id": "P101",
            # ❌ Missing 'name'
            "category": "Accessories",
            "price": 29.99,
            "features": ["Compact"]
        },
        {
            # ❌ Missing 'id'
            "name": "Missing ID Product",
            "category": "Wearables",
            "price": 59.99,
            "features": ["Lightweight"]
        },
        {
            "id": "P103",
            "name": "Wrong Type Price",
            "category": "Electronics",
            "price": "expensive",  # ❌ Wrong type
            "features": "Portable"  # ❌ Should be a list
        }
    ]
}

with open("invalid_products.json", "w", encoding="utf-8") as f:
    json.dump(invalid_products_data, f, indent=2)

print("invalid_products.json created successfully.")


invalid_products.json created successfully.


In [6]:
with open("malformed.json", "w", encoding="utf-8") as f:
    f.write("""
{
  "products": [
    {
      "id": "P001",
      "name": "Broken Product",
      "category": "Electronics",
      "price": 99.99,
      "features": ["Feature A", "Feature B"]
    }
  ]   // <-- Invalid comment in JSON
}
""")

print("malformed.json created successfully.")


malformed.json created successfully.


# STEP 1: Code Analysis

# Step-by-step Analysis of the Starter Code

## 1️⃣ What happens if `products.json` doesn't exist?

```python
with open(json_file, 'r') as f:
    data = json.load(f)
```

💥 If the file doesn't exist → `FileNotFoundError`

- ❌ No `try/except`
- ❌ No helpful message
- ❌ Program crashes immediately

> **DO NOT FAIL SILENTLY — FAIL AND SHOW WHERE**

---

## 2️⃣ What happens if JSON is invalid?

```python
data = json.load(f)
```

💥 If JSON is malformed → `json.JSONDecodeError`

- ❌ No handling
- ❌ Cryptic traceback
- ❌ No explanation of which file caused the issue

Manager would not be happy 😅

---

## 3️⃣ What happens if product validation fails?

```python
except:
    pass  # Silent failure!
```

🚨 Worst offender.

If:
- `price` is negative
- Required field missing
- Wrong type

👉 Product is silently ignored.

No:
- Error message
- Logging
- Count of skipped items
- Explanation

This creates **data corruption by silence**.

You could process 100 products and only generate 3… and never know why.

---

## 4️⃣ What happens if API call fails?

```python
response = client.chat.completions.create(...)
```

Possible failures:
- Invalid API key
- Rate limit
- Network issue
- Timeout
- Model not found

💥 Any of these → crash

- ❌ No retry
- ❌ No context
- ❌ No product ID in error message

You wouldn’t know which product caused it.

---

## 5️⃣ What functions are doing too much?

🚨 `generate_product_descriptions()`

It does:
- File loading
- JSON parsing
- Data validation
- API client creation
- Prompt creation
- API calling
- Response processing
- File writing
- Orchestration

Violates:

> **Single Responsibility Principle**

It’s a **God function**.

Impossible to:
- Unit test properly
- Reuse parts
- Debug easily
- Add logging cleanly

---

## 6️⃣ Where are concerns mixed together?

Everything is tightly coupled:

- Validation mixed with processing
- API client hardcoded
- File saving hardcoded to `"results.json"`
- Prompt logic inline
- No separation between orchestration and implementation

Future changes become painful.

If tomorrow you want:
- Different model
- Different output file
- Retry logic
- Logging

You must modify the main function directly.

---

## 7️⃣ Code Smells

### 🔴 Hardcoded API key

```python
client = OpenAI(api_key="your-api-key-here")
```

- Security issue
- Not production-ready
- Not environment-based

---

### 🔴 Hardcoded output file

```python
with open('results.json', 'w') as f:
```

- Not configurable

---

### 🔴 Mutable default in Pydantic model

```python
features: List[str] = []
```

Should be:

```python
Field(default_factory=list)
```

Subtle bug waiting to happen.

---

### 🔴 Broad `except:` without specifying error

Catches:
- `KeyboardInterrupt`
- `SystemExit`
- `ValidationError`

And hides them.

Very dangerous.

---

## 8️⃣ Missing Helper Functions

Should have:

- `load_json_file(path)`
- `validate_products(data)`
- `create_prompt(product)`
- `generate_description(client, product)`
- `save_results(results, path)`
- `initialize_openai_client()`

Each must do **one thing only**.

---

# ✅ Major Issues Identified

- ❌ No file error handling
- ❌ No JSON error handling
- ❌ Silent validation failures
- ❌ No API error handling
- ❌ Single giant function (SRP violation)
- ❌ Hardcoded API key
- ❌ Hardcoded output file
- ❌ No logging
- ❌ No retry logic
- ❌ Mutable default argument
- ❌ No context in error messages


# STEP 2: CREATE HELPER FUNCTIONS

## 1️⃣ load_json_file(file_path: str) -> dict

🎯 Responsibility

Only:
- Open file
- Parse JSON
- Handle file + JSON errors

Nothing else.

### What problems it solves

Instead of crashing silently, it should:

- Show WHAT went wrong
- Show WHERE (function name + file path)
- Show WHY (invalid JSON, file missing, etc.)

### What proper handling means

It should explicitly handle:

- FileNotFoundError
- json.JSONDecodeError
- Generic unexpected errors

### Example behavior

If file missing:

FileNotFoundError in load_json_file()  
File: products.json  
Reason: File does not exist.

If JSON invalid:

JSONDecodeError in load_json_file()  
File: products.json  
Line 12, Column 5  
Reason: Invalid JSON syntax.

Now you can test this function alone:

```python
load_json_file("missing.json")
```

It should fail clearly and loudly.

That’s the goal.

---

## 2️⃣ validate_product_data(product_dict: dict) -> Optional[Product]

🎯 Responsibility

Only:
- Take ONE product dictionary
- Validate it using Product
- Return valid Product OR raise meaningful error

### Why this matters

In the starter code:

```python
except:
    pass
```

That hides:
- Which product failed
- Which field failed
- Why it failed

Here, you want:

- Explicit ValidationError
- Show invalid fields
- Show product ID (if available)

### Example output

ValidationError in validate_product_data()  
Product ID: 123  
Field 'price': Price must be positive

Now you can test it independently:

```python
validate_product_data({"id": "1", "price": -10})
```

It should clearly explain what's wrong.

---

## 3️⃣ create_product_prompt(product: Product) -> str

🎯 Responsibility

Only:
- Create the prompt string

No API calls.  
No validation.  
No formatting.

### Why separate this?

Because prompt engineering evolves.

Tomorrow you may:
- Add tone
- Add SEO keywords
- Add character limits
- Use system role messages

If it's isolated, changing prompt logic doesn’t break API logic.

You can test:

```python
p = Product(...)
print(create_product_prompt(p))
```

Simple. Clean.

---

## 4️⃣ parse_api_response(response) -> str

🎯 Responsibility

Only:
- Extract the description text safely

The starter code does this blindly:

```python
description = response.choices[0].message.content
```

What if:
- choices empty?
- API changed format?
- Response malformed?

This helper should:
- Validate structure
- Raise meaningful error if malformed

### Example failure

ValueError in parse_api_response()  
Reason: API response has no choices.

Now API logic is isolated from business logic.

---

## 5️⃣ format_output(product: Product, description: str) -> dict

🎯 Responsibility

Only:
- Create final dictionary structure

```json
{
    "product_id": "...",
    "name": "...",
    "description": "..."
}
```

### Why separate this?

Because output format changes often.

Maybe tomorrow:
- Add timestamp
- Add category
- Add price
- Change JSON schema

This function becomes your formatting boundary.

---

## 🧠 What "I" Just Did Architecturally

I moved from:

- Big chaotic function

To:

- I/O  
  ↓  
- Validation  
  ↓  
- Prompt Creation  
  ↓  
- API Call  
  ↓  
- Response Parsing  
  ↓  
- Output Formatting  

That’s a pipeline.

Each step is:

- Testable
- Replaceable
- Observable
- Debuggable


## STEP 2.1 – Unit Test Each Function (Manual Try/Except Approach)

After building each helper function, I performed a manual unit test for every single one.

The goal was simple:  
Each helper must be tested independently to verify both success and failure behavior.

I did not test everything at once.  
I tested one helper at a time.

---

### Helper 1 — `load_json_file()`

After implementing the function, I ran isolated tests to confirm:

- It works correctly with a valid JSON file
- It raises a clear exception if the file does not exist
- It raises a clear exception if the JSON format is invalid

This verified:
- Proper file handling
- Proper JSON parsing
- Proper error reporting
- No silent failures

---

### Helper 2 — `validate_product_data()`

I created a unit test that:

- Confirms valid product data returns a proper `Product` object
- Confirms invalid data (e.g., negative price, missing fields) raises a clear validation error

This verified:
- Validation logic works
- Errors are explicit
- No product is silently ignored

---

### Helper 3 — `create_product_prompt()`

I tested that:

- The function correctly generates a prompt string
- The prompt contains the expected product information
- No external dependencies exist (no API calls, no validation logic inside)

This verified:
- Prompt generation is isolated
- Logic is deterministic
- The function is easy to modify without side effects

---

### Helper 4 — `parse_api_response()`

I tested:

- Correct extraction of the description from a valid API response structure
- Proper failure when the response structure is invalid (e.g., missing choices)

This verified:
- Defensive programming
- Clear error messaging
- API structure validation

---

### Helper 5 — `format_output()`

I confirmed that:

- The function returns the expected dictionary structure
- The mapping between product and description is correct
- No hidden logic is mixed into formatting

This verified:
- Output structure is clean and controlled
- Future schema changes can be isolated to one function

---

## 🧠 What This Demonstrates

For each helper:

- One isolated unit test
- Explicit expected behavior
- Explicit failure detection
- No silent corruption

Even though this was done manually (using controlled try/except testing instead of a formal testing framework), it demonstrates:

- Understanding of unit-level responsibility
- Positive and negative test coverage
- Controlled failure handling
- Clear architectural thinking

Each function now fails loudly, predictably, and in the right place.

That’s the core objective of Step 2.1.


## Helper 1: Load and parse JSON file with error handling.

- Explicit exception handling
- Clear error messages
- Shows file path
- Shows exact line/column for invalid JSON
- Keeps original traceback with from e
- No silent failure

In [9]:
import json

def load_json_file(file_path: str) -> dict:
    """Load and parse JSON file with error handling."""
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            return json.load(f)

    except FileNotFoundError as e:
        raise FileNotFoundError(
            f"File not found: '{file_path}'. Please check the file path."
        ) from e

    except json.JSONDecodeError as e:
        raise ValueError(
            f"Invalid JSON in file '{file_path}' "
            f"(line {e.lineno}, column {e.colno}): {e.msg}"
        ) from e




In [10]:
# TEST HELPER 1 — Load and Parse JSON

print("==========================================")
print("TESTING HELPER: load_json_file()")
print("==========================================\n")


# -----------------------------------
# Test Case 1: Valid JSON file
# Expected: No exception
# -----------------------------------
print("=== TEST CASE 1: VALID JSON START ===")

try:
    load_json_file("products.json")
    print("TEST OK")
except Exception:
    print("TEST FAILED")

print("=== TEST CASE 1: VALID JSON FINISH ===\n")


# -----------------------------------
# Test Case 2: File not found
# Expected: Exception
# -----------------------------------
print("=== TEST CASE 2: FILE NOT FOUND START ===")

try:
    load_json_file("missing_file.json")
    print("TEST FAILED: Error not detected")
except Exception:
    print("TEST OK: Error detected")

print("=== TEST CASE 2: FILE NOT FOUND FINISH ===\n")


# -----------------------------------
# Test Case 3: Invalid JSON format
# Expected: Exception
# -----------------------------------
print("=== TEST CASE 3: INVALID JSON START ===")

try:
    load_json_file("malformed.json")
    print("TEST FAILED: Error not detected")
except Exception:
    print("TEST OK: Error detected")

print("=== TEST CASE 3: INVALID JSON FINISH ===\n")


TESTING HELPER: load_json_file()

=== TEST CASE 1: VALID JSON START ===
TEST OK
=== TEST CASE 1: VALID JSON FINISH ===

=== TEST CASE 2: FILE NOT FOUND START ===
TEST OK: Error detected
=== TEST CASE 2: FILE NOT FOUND FINISH ===

=== TEST CASE 3: INVALID JSON START ===
TEST OK: Error detected
=== TEST CASE 3: INVALID JSON FINISH ===



## Helper 2: Validate product data using Pydantic.

- It catches only ValidationError
- It uses e.errors() (the structured Pydantic error output)
- It prints field name and specific reason
- It allows the batch process to continue
- It avoids bare except

In [11]:
from pydantic import ValidationError
from typing import Optional

def validate_product_data(product_dict: dict) -> Optional[Product]:
    """Validate product data using Pydantic."""
    try:
        return Product(**product_dict)

    except ValidationError as e:
        print("Product validation failed:")

        for error in e.errors():
            field = ".".join(str(loc) for loc in error["loc"])
            message = error["msg"]
            print(f" - Field '{field}': {message}")

        return None


In [12]:
# TEST HELPER 2 — Validate Product Data

print("==========================================")
print("TESTING HELPER: validate_product_data()")
print("==========================================\n")


# -----------------------------------
# Test Case 1: Valid products (should NOT return None)
# Expected: PASS
# -----------------------------------
print("=== TEST CASE 1: VALID PRODUCTS START ===")

try:
    data = load_json_file("products.json")
    all_valid = True

    for product_dict in data.get("products", []):
        result = validate_product_data(product_dict)
        if result is None:
            all_valid = False

    if all_valid:
        print("TEST OK")
    else:
        print("TEST FAILED")

except Exception:
    print("TEST FAILED")

print("=== TEST CASE 1: VALID PRODUCTS FINISH ===\n")


# -----------------------------------
# Test Case 2: Invalid products (should return None at least once)
# Expected: Error detected
# -----------------------------------
print("=== TEST CASE 2: INVALID PRODUCTS START ===")

try:
    data = load_json_file("invalid_products.json")
    error_detected = False

    for product_dict in data.get("products", []):
        result = validate_product_data(product_dict)
        if result is None:
            error_detected = True

    if error_detected:
        print("TEST OK: Error detected")
    else:
        print("TEST FAILED: Error not detected")

except Exception:
    print("TEST FAILED: Unexpected exception")

print("=== TEST CASE 2: INVALID PRODUCTS FINISH ===\n")


TESTING HELPER: validate_product_data()

=== TEST CASE 1: VALID PRODUCTS START ===
TEST OK
=== TEST CASE 1: VALID PRODUCTS FINISH ===

=== TEST CASE 2: INVALID PRODUCTS START ===
Product validation failed:
 - Field 'price': Value error, Price must be positive
Product validation failed:
 - Field 'name': Field required
Product validation failed:
 - Field 'id': Field required
Product validation failed:
 - Field 'price': Input should be a valid number, unable to parse string as a number
 - Field 'features': Input should be a valid list
TEST OK: Error detected
=== TEST CASE 2: INVALID PRODUCTS FINISH ===



## Helper 3: Generate OpenAI prompt for product.

- Prompt logic is isolated and reusable
- Easy to modify tone/style later
- Handles empty features safely
- Keeps formatting clean
- Makes your main orchestration function much simpler

In [ ]:
def create_product_prompt(product: Product) -> str:
    """Generate OpenAI prompt for product."""
    
    features_text = ", ".join(product.features) if product.features else "No specific features listed"
    
    prompt = (
        "Create a compelling product description for the following product:\n\n"
        f"Name: {product.name}\n"
        f"Category: {product.category}\n"
        f"Price: ${product.price:.2f}\n"
        f"Features: {features_text}\n\n"
        "The description should be persuasive, clear, and suitable for an online store."
    )
    
    return prompt


In [ ]:
# TEST HELPER 3 — Create Product Prompt

print("==========================================")
print("TESTING HELPER: create_product_prompt()")
print("==========================================\n")


# -----------------------------------
# Test Case 1: Product with features
# Expected: Valid non-empty string containing product name
# -----------------------------------
print("=== TEST CASE 1: PRODUCT WITH FEATURES START ===")

try:
    product_with_features = Product(
        id="201",
        name="Smartwatch",
        category="Wearables",
        price=199.99,
        features=["Heart rate monitor", "GPS", "Water resistant"]
    )

    prompt = create_product_prompt(product_with_features)

    if isinstance(prompt, str) and product_with_features.name in prompt:
        print("TEST OK")
    else:
        print("TEST FAILED")

except Exception:
    print("TEST FAILED")

print("=== TEST CASE 1: PRODUCT WITH FEATURES FINISH ===\n")


# -----------------------------------
# Test Case 2: Product without features
# Expected: Valid non-empty string without crashing
# -----------------------------------
print("=== TEST CASE 2: PRODUCT WITHOUT FEATURES START ===")

try:
    product_without_features = Product(
        id="202",
        name="Notebook",
        category="Stationery",
        price=4.99,
        features=[]
    )

    prompt = create_product_prompt(product_without_features)

    if isinstance(prompt, str) and product_without_features.name in prompt:
        print("TEST OK")
    else:
        print("TEST FAILED")

except Exception:
    print("TEST FAILED")

print("=== TEST CASE 2: PRODUCT WITHOUT FEATURES FINISH ===\n")


## Helper 4: Parse OpenAI API response

- Handles missing choices
- Handles empty list
- Handles missing message
- Handles empty content
- Raises a clear error instead of crashing randomly
- Preserves original traceback (from e)

In [ ]:
def parse_api_response(response) -> str:
    """Parse OpenAI API response."""
    try:
        content = response.choices[0].message.content
        
        if not content:
            raise ValueError("API response content is empty.")
        
        return content.strip()

    except (AttributeError, IndexError, KeyError) as e:
        raise ValueError("Unexpected API response structure.") from e


In [ ]:
# TEST HELPER 4 — Parse API Response

print("==========================================")
print("TESTING HELPER: parse_api_response()")
print("==========================================\n")


# -----------------------------------
# Test Case 1: Valid response with content
# Expected: Returns non-empty string
# -----------------------------------
print("=== TEST CASE 1: VALID RESPONSE START ===")

class FakeValidResponse:
    class Choice:
        class Message:
            content = "This is a generated product description."
        message = Message()
    choices = [Choice()]

valid_response = FakeValidResponse()

try:
    result = parse_api_response(valid_response)

    if isinstance(result, str) and result.strip() != "":
        print("TEST OK")
    else:
        print("TEST FAILED")

except Exception:
    print("TEST FAILED")

print("=== TEST CASE 1: VALID RESPONSE FINISH ===\n")


# -----------------------------------
# Test Case 2: Empty response content
# Expected: Exception
# -----------------------------------
print("=== TEST CASE 2: EMPTY CONTENT START ===")

class FakeEmptyContentResponse:
    class Choice:
        class Message:
            content = ""
        message = Message()
    choices = [Choice()]

empty_response = FakeEmptyContentResponse()

try:
    parse_api_response(empty_response)
    print("TEST FAILED: Error not detected")
except Exception:
    print("TEST OK: Error detected")

print("=== TEST CASE 2: EMPTY CONTENT FINISH ===\n")


# -----------------------------------
# Test Case 3: Unexpected response structure
# Expected: Exception
# -----------------------------------
print("=== TEST CASE 3: UNEXPECTED STRUCTURE START ===")

class FakeInvalidResponse:
    choices = []

invalid_response = FakeInvalidResponse()

try:
    parse_api_response(invalid_response)
    print("TEST FAILED: Error not detected")
except Exception:
    print("TEST OK: Error detected")

print("=== TEST CASE 3: UNEXPECTED STRUCTURE FINISH ===\n")


## Help 5: Format final output

- Keeps formatting logic separate from business logic
- Ensures description is clean (no trailing whitespace)
- Makes future changes easy (e.g., add price, category, timestamp, etc.)
- Makes main orchestration cleaner

In [7]:
def format_output(product: Product, description: str) -> dict:
    """Format final output."""
    return {
        "product_id": product.id,
        "name": product.name,
        "description": description.strip()
    }


In [8]:
# TEST HELPER 5 — Format Output

print("==========================================")
print("TESTING HELPER: format_output()")
print("==========================================\n")


# -----------------------------------
# Test Case 1: Normal product and description
# Expected: Correct dictionary formatting
# -----------------------------------
print("=== TEST CASE 1: NORMAL INPUT START ===")

try:
    product = Product(
        id="301",
        name="Desk Lamp",
        category="Home",
        price=29.99,
        features=["LED", "Adjustable Arm"]
    )

    description = "  A modern LED desk lamp.  "

    result = format_output(product, description)

    if (
        isinstance(result, dict) and
        result == {
            "product_id": "301",
            "name": "Desk Lamp",
            "description": "A modern LED desk lamp."
        }
    ):
        print("TEST OK")
    else:
        print("TEST FAILED")

except Exception:
    print("TEST FAILED")

print("=== TEST CASE 1: NORMAL INPUT FINISH ===\n")


TESTING HELPER: format_output()

=== TEST CASE 1: NORMAL INPUT START ===
TEST OK
=== TEST CASE 1: NORMAL INPUT FINISH ===



## STEP 3 – Modularize Functions

### Objective  
Break the original monolithic function into focused modules with clear responsibilities.

Instead of one large function doing everything, the logic is now divided into smaller functions, each responsible for one specific step of the pipeline.

---

## 1️⃣ `load_and_validate_products(json_path: str) -> List[Product]`

### What it does

This function is responsible for:

- Loading the JSON file
- Validating each product
- Returning a clean list of validated `Product` objects

It uses:
- `load_json_file()` for file loading and JSON parsing
- `validate_product_data()` for validating each product

### Why this matters

- File handling is separated from business logic.
- Validation errors are clearly tied to this stage.
- If something fails, you immediately know the problem is in loading or validation.

This function defines the **input boundary** of the system.

---

## 2️⃣ `generate_description(product: Product, api_client) -> str`

### What it does

This function:

- Creates the prompt for a single product
- Sends the request to the API
- Parses and returns the generated description

It uses:
- `create_product_prompt()` for prompt construction
- `parse_api_response()` for safely extracting the description

### Why this matters

- API logic is isolated.
- Prompt engineering changes won’t affect other parts of the system.
- API-related failures are clearly associated with this function.

This function defines the **AI boundary** of the system.

---

## 3️⃣ `process_products(products: List[Product], api_client) -> List[dict]`

### What it does

This function:

- Iterates over all validated products
- Calls `generate_description()` for each product
- Formats the output
- Handles errors per product (without stopping the entire process)

### Why this matters

- Orchestration is separated from implementation.
- One product failing does not crash the whole pipeline.
- Error handling becomes granular and observable.

This function defines the **processing pipeline layer**.

---

## 4️⃣ `save_results(results: List[dict], output_path: str) -> None`

### What it does

This function:

- Writes the final results to a JSON file
- Handles file-writing errors explicitly

### Why this matters

- Output handling is isolated.
- Future changes to output format or storage location affect only this function.
- File system errors are clearly tied to the saving stage.

This function defines the **output boundary** of the system.

---

## 🧠 Architectural Shift

You moved from:

One giant function doing everything

To a clean pipeline:

Load & Validate  
↓  
Generate Description  
↓  
Process Products  
↓  
Save Results  

Each function now:

- Has a single responsibility  
- Is testable in isolation  
- Fails in a predictable place  
- Can evolve independently  

---

## ✅ Checkpoint Achieved

The main function now:

- Only orchestrates the flow
- Does not contain business logic
- Does not mix concerns
- Does not perform detailed work

It simply coordinates the modules.

That’s proper modular design.


## FUNCTION 1: load_and_validate_products

## `load_and_validate_products(json_path: str) -> List[Product]`

### Purpose

This function loads product data from a JSON file and validates each product using the previously created helper functions.

It acts as a coordination layer between:

- `load_json_file()`
- `validate_product_data()`

---

### What It Does Step-by-Step

1. Loads the JSON file  
   - Uses `load_json_file(json_path)`  
   - If the file is missing or JSON is invalid, the error is raised with clear location details.

2. Verifies structure  
   - Ensures the `"products"` key exists.
   - Ensures `"products"` is a list.
   - Raises a clear error if structure is incorrect.

3. Validates each product  
   - Iterates through the product list.
   - Calls `validate_product_data()` for each item.
   - Valid products are added to the result list.
   - Invalid products are skipped with a clear message showing their index.

4. Returns validated products  
   - Only properly validated `Product` objects are returned.
   - Invalid entries do not stop execution.

---

### Why This Is Good Design

- Keeps responsibilities separated.
- Reuses helper functions.
- Clearly shows where errors occur.
- Prevents silent failures.
- Allows batch processing to continue even if some products are invalid.
- Easy to test independently.

---

It strictly handles loading and validation.


In [ ]:
from typing import List

def load_and_validate_products(json_path: str) -> List[Product]:
    """Load JSON and validate products."""
    
    # Load raw data (will raise clear errors if file missing or JSON invalid)
    data = load_json_file(json_path)

    if "products" not in data:
        raise ValueError(f"'products' key not found in '{json_path}'.")

    if not isinstance(data["products"], list):
        raise ValueError(f"'products' must be a list in '{json_path}'.")

    validated_products: List[Product] = []

    for index, product_dict in enumerate(data["products"], start=1):
        product = validate_product_data(product_dict)

        if product:
            validated_products.append(product)
        else:
            print(f"Product at index {index} in '{json_path}' is invalid and was skipped.")

    return validated_products


## FUNCTION 2: generate_description

## `generate_description(product: Product, api_client) -> str`

### Purpose

Generates a product description using the OpenAI API for a single validated `Product`.

This function coordinates:

- `create_product_prompt()`
- OpenAI API call
- `parse_api_response()`

---

### What It Does Step-by-Step

1. Creates the prompt  
   - Calls `create_product_prompt(product)`  
   - Keeps prompt logic separate from API logic  

2. Calls the API  
   - Sends the prompt using the provided `api_client`  
   - Uses the specified model  

3. Parses the API response  
   - Calls `parse_api_response(response)`  
   - Extracts and validates the returned content  

4. Handles errors clearly  
   - Wraps all API-related errors  
   - Raises a `RuntimeError` that includes:
     - Product name  
     - Product ID  
     - Original error message  
   - Preserves traceback for debugging  

---

### Why This Is Good Design

- Single responsibility: handles description generation only  
- Reuses helper functions  
- Shows exactly which product caused an API failure  
- Makes batch processing easier to manage  
- Easy to test independently (by mocking the API client)  

---

It strictly handles generating a description for one product.

In [16]:
def generate_description(product: Product, api_client) -> str:
    """Generate description for one product using API."""
    
    try:
        # Create prompt using helper
        prompt = create_product_prompt(product)

        # Call API
        response = api_client.chat.completions.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}]
        )

        # Parse response using helper
        return parse_api_response(response)

    except Exception as e:
        raise RuntimeError(
            f"API error while generating description for product "
            f"'{product.name}' (ID: {product.id}): {str(e)}"
        ) from e


## FUNCTION 3: process_products

## `process_products(products: List[Product], api_client) -> List[dict]`

### Purpose

Processes a list of validated `Product` objects and generates descriptions for each one.

This function acts as the orchestration layer for batch processing.

It coordinates:

- `generate_description()`
- `format_output()`

---

### What It Does Step-by-Step

1. Iterates through all validated products.
2. For each product:
   - Calls `generate_description()` to get the AI-generated text.
   - Calls `format_output()` to structure the final result.
   - Appends the formatted result to the results list.
3. If an error occurs for a product:
   - Prints a clear error message including product name and ID.
   - Skips that product.
   - Continues processing the remaining products.
4. Returns a list of successfully processed product dictionaries.

---

### Why This Is Good Design

- Handles errors per product instead of crashing the entire batch.
- Keeps orchestration separate from business logic.
- Makes the pipeline resilient.
- Easy to test by mocking the API client.
- Maintains clean separation of responsibilities.

---

It strictly coordinates processing for multiple products.


In [ ]:
from typing import List

def process_products(products: List[Product], api_client) -> List[dict]:
    """Process all products and generate descriptions."""
    
    results: List[dict] = []

    for product in products:
        try:
            description = generate_description(product, api_client)
            formatted_output = format_output(product, description)
            results.append(formatted_output)

        except Exception as e:
            print(
                f"Error processing product '{product.name}' "
                f"(ID: {product.id}): {str(e)}"
            )
            continue

    return results


## FUNCTION 4: save_results

## `save_results(results: List[dict], output_path: str) -> None`

### Purpose

Saves processed product results to a JSON file.

This function isolates file output logic from the rest of the application.

---

### What It Does Step-by-Step

1. Opens the specified output file in write mode.
2. Serializes the results list into formatted JSON.
3. Handles file-related errors clearly.
4. Raises a descriptive error including the output file path if saving fails.

---

### Why This Is Good Design

- Single responsibility: only handles saving data.
- Shows exactly where file errors occur.
- Preserves original traceback for debugging.
- Makes the main orchestration function cleaner.
- Easy to test independently.

---

It strictly handles persistence.


In [15]:
import json
from typing import List

def save_results(results: List[dict], output_path: str) -> None:
    """Save results to JSON file."""
    
    try:
        with open(output_path, "w", encoding="utf-8") as f:
            json.dump(results, f, indent=2, ensure_ascii=False)

    except OSError as e:
        raise OSError(
            f"File error while saving results to '{output_path}': {str(e)}"
        ) from e


## MAIN FUNCTION

## `main()`

### Purpose

The `main` function acts purely as the orchestration layer of the application.

It coordinates the overall workflow without implementing business logic directly.

---

### What It Does Step-by-Step

1. Configures the API client  
   - Reads the API key from environment variables.  
   - Initializes the OpenAI client.  

2. Loads and validates products  
   - Calls `load_and_validate_products()`.  
   - Stops early if no valid products exist.  

3. Processes products  
   - Calls `process_products()` to generate descriptions.  

4. Saves results  
   - Calls `save_results()` to persist output to file.  

5. Handles top-level errors  
   - Catches application-level exceptions.  
   - Displays a clear message if something fails.  

---

### Why This Is Correct Modular Design

- The function is short and readable.
- It describes *what* happens, not *how* it happens.
- It delegates all real work to helper functions.
- It does not contain validation logic.
- It does not contain API parsing logic.
- It does not contain formatting logic.
- It does not contain file handling logic beyond orchestration.

---

### Structural Outcome

The program now follows a clean pipeline:

1. Load  
2. Validate  
3. Generate  
4. Format  
5. Save  

The `main` function simply connects the modules together.


In [14]:
import os
from openai import OpenAI


def main():
    """Main orchestration function."""

    try:
        # Configure API client
        api_key = os.getenv("OPENAI_API_KEY")
        if not api_key:
            raise EnvironmentError("OPENAI_API_KEY environment variable is not set.")

        api_client = OpenAI(api_key=api_key)

        # Load and validate products
        products = load_and_validate_products("products.json")

        if not products:
            print("No valid products to process.")
            return

        # Process products
        results = process_products(products, api_client)

        # Save results
        save_results(results, "results.json")

        print(f"Successfully processed {len(results)} product(s).")

    except Exception as e:
        print(f"Application error: {str(e)}")


if __name__ == "__main__":
    main()


Application error: name 'load_and_validate_products' is not defined


## STEP 4 – Add Error Handling (CRITICAL)

### Objective  
Add comprehensive error handling that clearly shows **WHERE** errors occur, instead of crashing silently or producing cryptic tracebacks.

The goal is structured, explicit, helpful error messages.

---

## 1️⃣ FileNotFoundError Handling

### What this does

Inside `load_json_file()`, the file opening logic is wrapped in a `try` block.  
If the file does not exist, the function:

- Catches `FileNotFoundError`
- Builds a structured error message
- Shows:
  - The function name
  - The missing file path
  - The current working directory
  - A suggestion to fix it
- Prints the message
- Re-raises the exception

### Why this matters

Instead of a raw Python traceback, the user sees:

- Where the error happened (`load_json_file`)
- What file was missing
- Why it failed
- What to check next

Re-raising the exception preserves correct program behavior while making the error understandable.

---

## 2️⃣ JSONDecodeError Handling

### What this does

When JSON parsing fails, the code catches `json.JSONDecodeError`.

It then:

- Extracts the line number and column from the exception object
- Displays:
  - File name
  - Exact location (line and column)
  - The JSON parser message
  - A suggestion to fix the syntax
- Prints the structured message
- Re-raises the exception

### Why this matters

Malformed JSON errors can be hard to debug.

Now the error clearly shows:

- Which file is broken
- Where inside the file
- What kind of syntax issue occurred

This transforms a technical traceback into an actionable debugging message.

---

## 3️⃣ Pydantic ValidationError Handling

### What this does

Inside `validate_product_data()`:

- The product dictionary is validated using the `Product` model.
- If validation fails, a `ValidationError` is caught.
- A structured error message is built that includes:
  - Function name
  - Product ID (if available)
  - A list of invalid fields
  - The reason each field failed validation
  - A suggestion to fix the fields

Instead of crashing, the function returns `None`.

### Why this matters

In the original version, validation failures were silently ignored.

Now:

- You know exactly which product failed.
- You know exactly which fields are invalid.
- You know why validation failed.
- No silent data corruption occurs.

Returning `None` allows higher-level orchestration logic to decide how to handle invalid products.

---

## 4️⃣ OpenAI APIError Handling

### What this does

Inside `generate_description()`:

- The API call is wrapped in a `try` block.
- If an `APIError` occurs, the function:
  - Identifies the product involved
  - Shows the error type
  - Shows the status code (if available)
  - Shows the raw error message
  - Suggests checking API key, rate limits, or retrying
- Prints the structured error
- Re-raises the exception

### Why this matters

API failures can come from:

- Invalid API keys
- Rate limits
- Model issues
- Server errors

Now the error message includes:

- Which product triggered it
- What kind of API error occurred
- Whether a status code is available
- A helpful suggestion

This makes API debugging far easier and more professional.

---

## 5️⃣ Network Error Handling

For network-related failures (timeouts, connection errors), the same structured format is used:

- Identify the function
- Identify the context (which product or file)
- Show the exact error message
- Provide a helpful suggestion

This ensures consistency across all failure types.

---

## 🧱 Standardized Error Message Format

All errors now follow a consistent structure:

ERROR in {function_name}(): {error_type}  
Location: {context}  
Message: {error_message}  
Suggestion: {helpful_tip}

### Why this matters

Consistency improves:

- Debugging speed
- Readability
- Maintainability
- Professionalism

You can immediately see:
- Where the failure occurred
- What went wrong
- What to do next

---

## ✅ Expected Outcome

After these improvements:

- No silent failures
- No hidden validation errors
- No ambiguous tracebacks
- All errors clearly show WHERE they occurred
- All errors include actionable suggestions

---



## Updated `load_json_file()` — Enhanced Error Handling

### Purpose

This updated version improves error handling to clearly indicate:

- Where the error occurred  
- Why the error occurred  
- What action might fix it  

It satisfies the lab requirement of showing precise error context.

---

### Error Cases Covered

1. File Not Found  
   - Shows the missing file path  
   - Shows the current working directory  
   - Suggests checking the file path  

2. Invalid JSON  
   - Shows the file name  
   - Shows the exact line and column of the error  
   - Displays the JSON parsing issue  
   - Suggests validating the JSON format  

3. Unexpected Errors  
   - Captures any other unexpected exception  
   - Displays the problem clearly  
   - Preserves traceback for debugging  

---

This ensures comprehensive and transparent error reporting.


In [13]:
import json
import os


def load_json_file(file_path: str) -> dict:
    """Load and parse JSON file with comprehensive error handling."""
    function_name = "load_json_file"

    try:
        with open(file_path, "r", encoding="utf-8") as f:
            return json.load(f)

    except FileNotFoundError as e:
        error_msg = (
            f"ERROR in {function_name}(): {type(e).__name__}\n"
            f"  Location: File '{file_path}' not found\n"
            f"  Message: {str(e)}\n"
            f"  Suggestion: Check that the file path is correct "
            f"(Current directory: {os.getcwd()})"
        )
        print(error_msg)
        raise

    except json.JSONDecodeError as e:
        error_msg = (
            f"ERROR in {function_name}(): {type(e).__name__}\n"
            f"  Location: File '{file_path}', line {e.lineno}, column {e.colno}\n"
            f"  Message: {e.msg}\n"
            f"  Suggestion: Check JSON syntax at line {e.lineno}"
        )
        print(error_msg)
        raise

    except Exception as e:
        error_msg = (
            f"ERROR in {function_name}(): {type(e).__name__}\n"
            f"  Location: File '{file_path}'\n"
            f"  Message: {str(e)}\n"
            f"  Suggestion: Check the file content and permissions"
        )
        print(error_msg)
        raise



## Updated `validate_product_data()` — Enhanced Validation Error Handling

### Purpose

This updated version improves validation error handling to clearly indicate:

- Where the error occurred  
- Which product caused the error  
- Which fields are invalid  
- Why those fields are invalid  
- What action might fix it  

It satisfies the lab requirement of showing precise validation context.

---

### Error Cases Covered

1. Pydantic ValidationError  
   - Shows the function name where validation failed  
   - Displays the Product ID (or "unknown" if missing)  
   - Lists each invalid field  
   - Explains why each field failed validation  
   - Suggests fixing the listed fields  
   - Returns `None` so batch processing can continue  

---

This ensures comprehensive and transparent validation error reporting without stopping the entire pipeline.


In [ ]:
from pydantic import ValidationError
from typing import Optional


def validate_product_data(product_dict: dict) -> Optional[Product]:
    """Validate product data using Pydantic with structured error reporting."""
    function_name = "validate_product_data"

    try:
        return Product(**product_dict)

    except ValidationError as e:
        product_id = product_dict.get("id", "unknown")

        # Build detailed field error messages
        field_errors = []
        for error in e.errors():
            field = ".".join(str(loc) for loc in error["loc"])
            message = error["msg"]
            field_errors.append(f"{field}: {message}")

        combined_message = "; ".join(field_errors)

        error_msg = (
            f"ERROR in {function_name}(): {type(e).__name__}\n"
            f"  Location: Product ID '{product_id}'\n"
            f"  Message: {combined_message}\n"
            f"  Suggestion: Fix the invalid fields listed above"
        )

        print(error_msg)
        return None


## Updated `create_product_prompt()` — Defensive Prompt Generation

### Purpose

This updated version improves prompt generation reliability by:

- Clearly defining the function boundary (prompt creation only)
- Safely handling missing or empty features
- Catching unexpected attribute errors
- Showing exactly where prompt generation failed
- Providing actionable debugging suggestions

It ensures that prompt creation failures are explicit and traceable.

---

### Normal Behavior

When valid product data is provided, the function:

- Extracts product attributes (`name`, `category`, `price`, `features`)
- Converts the features list into a readable string
- Falls back to a default message if no features are provided
- Builds a structured, persuasive prompt
- Returns the final prompt string

This keeps prompt logic isolated and clean.

---

### Error Handling Improvements

The function wraps the prompt construction in a `try` block to guard against unexpected issues such as:

- Missing product attributes
- Incorrect data types
- Corrupted product object
- Formatting errors (e.g., invalid price value)

If an exception occurs, it:

- Identifies the function where the failure happened
- Displays the product ID (or `"unknown"` if unavailable)
- Specifies the type of exception raised
- Provides a clear explanation of what failed
- Suggests verifying required product attributes
- Raises a structured `ValueError` to maintain controlled failure

---

### Error Cases Covered

1. AttributeError  
   - If the product object is missing required attributes  
   - Clearly indicates invalid or incomplete product data  

2. TypeError  
   - If data types are incorrect (e.g., price not numeric)  

3. Any Unexpected Exception  
   - Captured generically  
   - Wrapped in a controlled `ValueError`  
   - Prevents silent corruption  

---

### Why This Matters

Prompt generation is part of the AI boundary.  
If this step fails silently:

- API calls could receive malformed prompts
- Debugging becomes difficult
- Downstream errors become misleading

With this structure:

- Failures are explicit
- Errors show context
- The pipeline remains debuggable
- Responsibility remains single and isolated

---

This ensures clear, structured, and transparent prompt-generation error handling while preserving single responsibility.


In [ ]:
def create_product_prompt(product: Product) -> str:
    """Generate OpenAI prompt for product."""
    function_name = "create_product_prompt"

    try:
        features_text = ", ".join(product.features) if product.features else "No specific features listed"

        prompt = (
            "Create a compelling product description for the following product:\n\n"
            f"Name: {product.name}\n"
            f"Category: {product.category}\n"
            f"Price: ${product.price:.2f}\n"
            f"Features: {features_text}\n\n"
            "The description should be persuasive, clear, and suitable for an online store."
        )

        return prompt

    except Exception as e:
        error_msg = (
            f"ERROR in {function_name}(): {type(e).__name__}\n"
            f"  Location: Product '{getattr(product, 'id', 'unknown')}'\n"
            f"  Message: Failed to generate prompt due to invalid product data\n"
            f"  Suggestion: Ensure the product object contains valid attributes "
            f"(name, category, price, features)"
        )
        print(error_msg)
        raise ValueError(error_msg) from e


## Updated `generate_description()` — Enhanced API Error Handling

### Purpose

This updated version improves API error handling to clearly indicate:

- Where the error occurred  
- Which product caused the error  
- What type of API error occurred  
- What the error message says  
- What action might fix it  

It satisfies the lab requirement of showing precise API error context.

---

### Error Cases Covered

1. OpenAI APIError  
   - Shows the function name where the API call failed  
   - Displays the Product name and ID  
   - Identifies the specific API error type  
   - Shows the error message returned by the API  
   - Suggests checking API key, rate limits, network issues, or retrying later  
   - Re-raises the exception to preserve traceback  

2. Unexpected Exceptions  
   - Captures any other unexpected error during prompt creation, API call, or response parsing  
   - Shows the product context  
   - Displays the exception type and message  
   - Suggests checking response structure or internal logic  
   - Re-raises the exception  

---

This ensures comprehensive and transparent API error reporting while preserving debugging traceability.


In [ ]:
from openai import APIError


def generate_description(product: Product, api_client) -> str:
    """Generate description for one product using API with structured error handling."""
    function_name = "generate_description"

    try:
        # Create prompt
        prompt = create_product_prompt(product)

        # API call
        response = api_client.chat.completions.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}]
        )

        # Parse response using existing helper
        return parse_api_response(response)

    except APIError as e:
        error_msg = (
            f"ERROR in {function_name}(): {type(e).__name__}\n"
            f"  Location: Product '{product.name}' (ID: {product.id})\n"
            f"  Message: {str(e)}\n"
            f"  Suggestion: Check API key, rate limits, network connection, or try again later"
        )
        print(error_msg)
        raise

    except Exception as e:
        error_msg = (
            f"ERROR in {function_name}(): {type(e).__name__}\n"
            f"  Location: Product '{product.name}' (ID: {product.id})\n"
            f"  Message: {str(e)}\n"
            f"  Suggestion: Check API response structure or internal processing logic"
        )
        print(error_msg)
        raise


## Updated `parse_api_response()` — Defensive API Response Parsing

### Purpose

This updated version strengthens API response handling by:

- Safely extracting the generated content
- Validating that the content is not empty
- Detecting malformed response structures
- Clearly reporting where and why parsing failed
- Preventing silent downstream errors

It ensures that API response parsing is robust and transparent.

---

### Normal Behavior

When the API response structure is valid, the function:

- Accesses `choices[0].message.content`
- Verifies that content exists and is not empty
- Removes leading/trailing whitespace
- Returns the cleaned description string

This keeps the function focused strictly on parsing logic.

---

### Error Handling Improvements

The function is wrapped in a `try` block to guard against structural issues in the API response.

It explicitly checks:

1. That `choices` exists  
2. That at least one choice is present  
3. That `message.content` exists  
4. That the content is not empty  

If the content is empty:

- A structured `ValueError` is raised
- The error clearly states that the model returned no content
- A suggestion is provided to verify model behavior

---

### Error Cases Covered

1️⃣ Empty Content  
- Detected explicitly  
- Raises a `ValueError`  
- Suggests verifying model output  

2️⃣ AttributeError  
- Occurs if expected attributes are missing  

3️⃣ IndexError  
- Occurs if `choices` is empty  

4️⃣ KeyError  
- Occurs if the response structure differs from expectations  

In structural failure cases, the function:

- Identifies the exact parsing location (`choices[0].message.content`)
- Explains that the structure is malformed
- Suggests verifying the API response format
- Raises a controlled `ValueError` to stop unsafe execution

---

### Why This Matters

API responses are external inputs and therefore unreliable.

Without defensive parsing:

- A malformed response could crash the system unexpectedly
- Errors could appear far from their true source
- Debugging would be significantly harder

With this structure:

- Parsing errors are isolated
- Failures are explicit and contextual
- The pipeline remains predictable and debuggable
- The system fails loudly and correctly

---

This ensures structured, defensive, and production-grade API response handling while preserving single responsibility.


In [ ]:
def parse_api_response(response) -> str:
    """Parse OpenAI API response."""
    function_name = "parse_api_response"

    try:
        content = response.choices[0].message.content

        if not content:
            error_msg = (
                f"ERROR in {function_name}(): ValueError\n"
                f"  Location: API response content\n"
                f"  Message: API response content is empty\n"
                f"  Suggestion: Verify that the model returned a valid completion"
            )
            print(error_msg)
            raise ValueError(error_msg)

        return content.strip()

    except (AttributeError, IndexError, KeyError) as e:
        error_msg = (
            f"ERROR in {function_name}(): {type(e).__name__}\n"
            f"  Location: API response structure (choices[0].message.content)\n"
            f"  Message: Unexpected or malformed response structure\n"
            f"  Suggestion: Ensure the API response contains 'choices[0].message.content'"
        )
        print(error_msg)
        raise ValueError(error_msg) from e


## STEP 5 – Test Your Refactored Code

### Objective  
Verify that the fully refactored and modularized system behaves correctly under both normal and failure conditions.

---

## What Was Done

To validate the final architecture, a new clean Jupyter Notebook was created:

**`lab202refactoring_clean.ipynb`**

This notebook was used as a controlled integration testing environment to execute the complete pipeline from start to finish.

In addition, a proper `main()` function was created and placed at the correct orchestration level.  
The main function now:

- Coordinates the workflow
- Calls modular functions in sequence
- Does not contain business logic
- Does not mix concerns
- Serves purely as the system entry point

---

## 1️⃣ Clean Execution Environment

- A fresh notebook ensured no hidden state from previous executions.
- All helpers and modular functions were loaded cleanly.
- The complete pipeline was executed end-to-end.

This confirms the system runs independently and reproducibly.

---

## 2️⃣ Modular Error Handling Verified

Error handling is implemented at every architectural layer:

- File errors handled during loading
- Validation errors handled during product validation
- API errors handled during description generation
- File write errors handled during saving

Each function reports:
- Where the error occurred
- What failed
- Why it failed
- What action to take

No silent failures remain.

---

## 3️⃣ Full Integration Testing

Instead of testing only isolated helpers, the full pipeline was executed:

Load → Validate → Generate → Parse → Format → Save

This verifies:

- Proper interaction between modules
- Clean data flow between layers
- No responsibility overlap
- Controlled error propagation

The system behaves as a structured pipeline rather than a script.

---

## 4️⃣ Persistent Integration Logging

Integration test results were logged to a dedicated file:

**`integration_test_results.txt`**

This file contains:

- Success confirmations
- Structured error messages
- Contextual failure information
- Execution trace visibility

This ensures:

- Results are persistent
- Debugging does not rely solely on console output
- System behavior can be reviewed after execution
- Testing is documented and traceable

---

## Scenarios Tested

The integration process covered:

- Valid product data
- Missing input file
- Malformed JSON
- Invalid product fields
- API-related failures
- Output file writing errors

Each scenario produced:

- Clear, structured messages
- No silent data loss
- No uncontrolled crashes
- No ambiguous tracebacks

---

## Architectural Outcome

At this stage:

- The `main()` function strictly orchestrates.
- Each module has a single responsibility.
- Errors clearly show WHERE they occurred.
- Failures are explicit and controlled.
- The system is predictable under invalid input conditions.

---

## ✅ Checkpoint Achieved

The refactored system is now:

- Modular  
- Testable  
- Observable  
- Robust  
- Architecturally clean  

The integration testing, supported by `integration_test_results.txt`, confirms that the system works end-to-end under both normal and failure scenarios.
